# Setup
1. Install and import packages
2. Declare variables and set connection strings

In [ ]:
# install packages
print("Installing Necesssary Packages, please wait.")
import sys
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install datetime
!{sys.executable} -m pip install seaborn
!{sys.executable} -m pip install azure-kusto-data
print("Package installation complete.")

In [ ]:
print("Importing Packages")
import pandas as pd
import seaborn as sns
import os
import datetime
import azure.kusto.data
from datetime import timedelta
from azure.kusto.data import KustoClient, KustoConnectionStringBuilder, ClientRequestProperties
from azure.kusto.data.exceptions import KustoServiceError
from azure.kusto.data.helpers import dataframe_from_result_table
print("Package Import complete")

In [ ]:
#Adding IOCs as Global Variables to be used across functions
badguys = ()
compromised_accounts = "('User1','User2')"
compromised_machines = "('Computer1', 'Computer2')"
start_time = "ago(2d)"
end_time = "ago(1d)"
privesc_tools = ("winPEAs", "adPEAs", "mimikatz", "powerup", "sherlock", "seatbelt", "jaws-enum", "wes.py", "privesccheck")
tool_used = ()
attribution = ()

In [ ]:
cluster = "https://yourwindowseventlogcluster.net" 
cdoc_Conn = KustoConnectionStringBuilder.with_az_cli_authentication(cluster) 

client = KustoClient(events_Conn)
database = "WEC"

# Queries

In [ ]:
# starting point
initial_wec = '''
SecurityLog
| where ingestion_time() between('''+start_time + ".." + end_time+''')
| where EventId == 4688
| where EventData.TargetUserName has_any '''+compromised_accounts+'''
| extend SubjectUserName = tostring(parse_json(EventData).SubjectUserName),
TargetUserName = tostring(parse_json(EventData).TargetUserName),
CommandLine = tostring(parse_json(EventData).CommandLine),
ParentProcessName = tostring(parse_json(EventData).ParentProcessName),
NewProcessName = tostring(parse_json(EventData).NewProcessName)
| project TimeCreated, Computer, SubjectUserName, TargetUserName, ParentProcessName, NewProcessName, CommandLine
| limit 1000'''


InitialWEC_Results = client.execute(database, initial_wec)
InitialWEC_df = dataframe_from_result_table(InitialWEC_Results.primary_results[0])
#showing top 10 results to make sure you have the columns you need
#you can print out the whole thing but the idea was to then run the analytics on the results to pull exactly what you want
# by using pandas and making these into DFs, the analysis should be smoother/faster
InitialWEC_df.head(10)


In [ ]:
# the %1937 tokenelevation type indicates a user running with admin or equivalent permissions, 
# similar to executing with runas
token_elevation = '''
SecurityLog
| where ingestion_time() between('''+start_time + ".." + end_time+''')
| where EventId == 4688
| where Computer contains "JMP"
| where EventData.TokenElevation.Type contains "1937"
| sort by TimeCreated desc
| limit 1000
'''

token_elevation_results = client.execute(database, token_elevation)
token_elevation_df = dataframe_from_result_table(token_elevation_results.primary_results[0])
token_elevation_df.head(10)

In [ ]:
# suspicious rundll32, this is a technique of dll hijacking
rundll_hijacking = '''
SecurityLog
| where ingestion_time() between('''+start_time + ".." + end_time+''')
| where EventId == 4688
| where EventData.CommandLine contains "rundll32"
| where EventData.NewProcessName !contains "system32"
| sort by TimeCreated desc
| limit 1000
'''

rundll_results = client.execute(database, rundll_hijacking)
rundll_df = dataframe_from_result_table(rundll_results.primary_results[0])
rundll_df.head(10)

In [ ]:
# querying for possible pass the hash techniques - still working on this
pass_the_hash = '''
SecurityLog
| where ingestion_time() between('''+start_time + ".." + end_time+''')
| where EventId == 4624 and EventData.LogonType == 9
| where isnotempty(EventData.TargetOutBoundUserName) and EventData.TargetOutBoundUserName != "-"
| project TimeCreated, Computer, EventData.TargetOutBoundUserName, EventData.LogonType, EventData.IpAddress EventData.LogonGuid 
| join (SecurityLog
        | where TimeCreated between(start_time..end_time)
        | where EventId == 4648
        | where EventData.TargetServerName != "-" and EventData.TargetServerName != "localhost"
        | project TimeCreated, EventData.TargetServerName, Computer, EventData.TargetUserName, EventData.TargetInfo,
        EventData.LogonGiud
        ) on EventData.LogonGuid
| sort by TimeCreated desc
'''

pth_results = client.execute(database, pass_the_hash)
pth_df = dataframe_from_result_table(pth_results.primary_results[0])
pth_df.head(10)

In [ ]:
# user logins
user_login = '''
SecurityLog
| where ingestion_time() between('''+start_time + ".." + end_time+''')
| where EventId == 4624
| where EventData.TargetUserName has_any '''+compromised_accounts+'''
| extend TargetUserName = tostring(EventData.TargetUserName),
    TargetUserSid = EventData.TargetUserSid,
    IpAddress = tostring(EventData.IpAddress),
    LogonType = tostring(EventData.LogonType)
| where TargetUserSid !~ 'S-1-5-18'
| summarize Count=count() by Computer,TargetUserName, IpAddress, LogonType
| sort by Count desc
'''

user_login_results = client.execute(database, user_login)
user_login_df = dataframe_from_result_table(user_login_results.primary_results[0])
user_login_df.head(10)

### Visualization of User Logins

In [ ]:
sns.barplot(x = 'TargetUserName',
            y = 'Count',
            hue = 'Computer',
            data = user_login_df)
plt.gcf().set_size_inches(10,10)
plt.legend(bbox_to_anchor=(1.05, 0.5), loc='upper left', borderaxespad=0)
plt.show()

In [ ]:
#logins by machine
computer_login = '''
SecurityLog
| where ingestion_time() between('''+start_time + ".." + end_time+''')
| where EventId == 4624
| where Computer has_any '''+compromised_machines+'''
| extend TargetUserName = tostring(EventData.TargetUserName),
    TargetUserSid = EventData.TargetUserSid,
    IpAddress = tostring(EventData.IpAddress),
    LogonType = tostring(EventData.LogonType)
| where TargetUserSid !~ 'S-1-5-18'
| summarize Count=count() by TargetUserName, Computer, IpAddress, LogonType
| sort by Count desc
'''

computer_login_results = client.execute(database, computer_login)
computer_login_df = dataframe_from_result_table(computer_login_results.primary_results[0])
computer_login_df.head(10)

### Visualization of Computer Logins

In [ ]:
sns.barplot(x = "Computer",
            y = "Count",
            hue = "TargetUserName",
            data = computer_login_df)
plt.gcf().set_size_inches(10,10)
plt.legend(bbox_to_anchor=(1.05, 0.5), loc='upper left', borderaxespad=0)
plt.show()  

# Data Analytics

In [ ]:
#all unique commandline arguments
for cmd in InitialWEC_df['CommandLine'].unique():
    print(cmd +"\n")
    

In [ ]:
#find any powershell encoded commands
encoded_filter = InitialWEC_df['CommandLine'].str.find("encodedcommand")
encoded_filter

In [ ]:
#number of processes spawned by each parent
#this is something we could baseline as well - notate the values over the course of a normal week for example and
proc_relationship = InitialWEC_df.groupby(['ParentProcessName']).size()
proc_relationship